In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import clear_output

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    print('GPU device not found')
print('Found GPU at: {}'.format(device_name))
tf.config.threading.set_intra_op_parallelism_threads(0)

from tensorflow.keras import Sequential, Input, initializers, optimizers, callbacks, layers

from tensorflow.keras.layers import AveragePooling2D, BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPool2D
from tensorflow.keras.metrics import AUC

from tensorflow.keras.utils import to_categorical

In [ ]:
!pip install py7zr
from py7zr import unpack_7zarchive
import shutil
shutil.register_unpack_format('7zip', ['.7z'], unpack_7zarchive)
shutil.unpack_archive('/kaggle/input/cifar-10/train.7z', '/kaggle/working')
shutil.unpack_archive('/kaggle/input/cifar-10/test.7z', '/kaggle/working')

In [ ]:
train_dir = os.listdir("./train");
train_dir_len = len(train_dir)
print(".\\train:\t",train_dir_len)
print("files:\t\t",train_dir[:3])

In [ ]:
test_dir = os.listdir("./test");
test_dir_len = len(test_dir)
print(".\\test:\t",test_dir_len)
print("files:\t\t",test_dir[:3])

In [ ]:
import imageio

def read_images(path, number):
    a = np.concatenate([imageio.imread(path+f'/{i}.png')[None,...] for i in range(1,number+1)], axis=0)
    return a

In [ ]:
import pandas as pd
train_labels = pd.read_csv('/kaggle/input/cifar-10/trainLabels.csv', index_col=0)
train_labels = pd.get_dummies(train_labels, prefix='', prefix_sep='')
labels = train_labels.columns
y_train = train_labels.values

X_train = read_images("./train", train_dir_len)

In [ ]:
fig, axs = plt.subplots(2, 5, figsize=(11, 5))
fig.suptitle(f"Number of classes: {len(labels)}", fontsize=16)

for i, ax in enumerate(axs.flatten()):
    ind = np.random.choice(np.nonzero(y_train[:, i])[0])
    ax.imshow(X_train[ind])
    ax.set_title(f'{labels[i]} ({i})')
    ax.axis('off')

In [ ]:
data_augmentation = Sequential(
  [
      layers.experimental.preprocessing.RandomFlip("horizontal"),
      layers.experimental.preprocessing.RandomZoom(0.1),
      layers.experimental.preprocessing.RandomRotation(0.15),
      layers.experimental.preprocessing.RandomContrast(0.2)
  ],
  name='RandomAugmentator'
)

model = Sequential(
  [
    Input((32, 32, 3)),
    data_augmentation,
   
    Conv2D(48, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(48, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPool2D((2, 2)),
    Dropout(0.2),

    Conv2D(96, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(96, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPool2D((2, 2)),
    Dropout(0.3),

    Conv2D(192, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(192, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    AveragePooling2D((3, 3), strides=(2, 2)),
    Dropout(0.4),

    Flatten(),
    Dense(192, activation='relu'),
    Dropout(0.4),
    Dense(10, activation='softmax')
  ],
  name="CIFAR10-model"
)

model.summary()

In [ ]:
model.compile(
    optimizer=optimizers.Adam(0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
    )

checkpoint_filepath = '/kaggle/temp/checkpoint'
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
    )

early_stopping = callbacks.EarlyStopping(monitor='val_loss', min_delta=0,
                                         patience=20, verbose=1, mode='auto',
                                         restore_best_weights=True)


history = model.fit(X_train, y_train, epochs=400, initial_epoch=0, validation_split=0.1,
                    callbacks=[model_checkpoint_callback, early_stopping]
                    )

model.save('./model')

In [ ]:
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='validation')
plt.legend();

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend();

In [ ]:
X_test = read_images("./test", test_dir_len)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
labels_pred = pd.Series(y_pred.argmax(axis=1), index=pd.RangeIndex(1, test_dir_len+1)).apply(lambda x: labels[x])
labels_pred.index.name = 'id'
labels_pred.name = 'label'
labels_pred.head()

In [ ]:
fig, axs = plt.subplots(2, 5, figsize=(11, 5))
fig.suptitle(f"Test set predictions", fontsize=16)

for i, ax in enumerate(axs.flatten()):
    ax.imshow(X_test[i])
    ax.set_title(labels_pred.iloc[i])
    ax.axis('off')

In [ ]:
labels_pred.to_csv('submission.csv')